# 1. 어떤 기준으로 모델을 학습할 것인지 선택

- 참고하여 Layer 수 선정을 진행

[The Number of Hidden Layers | Heaton Research](https://web.archive.org/web/20140721050413/http://www.heatonresearch.com/node/707)

<aside>
💡 **Baseline** - 기준이 될 조건 정의/명세

- **Problem type** - Regression, Data type - 2 (Suggested)
- **Epoch** : Output - Train/Test loss 과적합 개형 보일 떄
- **Learning Rate** - 0.03
- **Activation** - ReLU
- **~~Regularization** - None / rate = 0~~
- **Ratio of training to test**
    - data = 70%
    - Noise = 10
    - Batch size = 10
- **Features** : X1, X2
    - Layer : 2개 -> **증가/감소시킬 때 Default 2, 5개**
- **Discretize output** : Off
</aside>

- Baseline 결과 (참고, 비교용)

| Loss | Train | Test |
| --- | --- | --- |
| Baseline | 0.027 | 0.028 |



# 2. features에서 두 그룹을 확실히 나눌 수 있는 조합과, 나누지 못하는 조합을 찾아봅니다.

<aside>
💡 전체 7개의 피쳐[$X_1, X_2, X_1^2, X_2^2, X_1X_2, sin(X_1), sin(X_2)$]((2^7) - 1 = 127개의 경우의 수)를 다 해보기에는 경우의 수가 많음. 
- 단계적으로 진행

</aside>

## 1개 선택 : $X_1, X_1^2, sin(X_1)$


| Loss | Train | Test |
| --- | --- | --- |
| $X_1$ | 0.044 | 0.042 |
| $X_1^2$ | 0.045 | 0.042 |
| $sin(X_1)$ | 0.045 | 0.042 |
| Baseline($X_1, X_2$) | 0.027 | 0.028 |

한가지 피쳐만을 쓰는 것은 Baseline $X_1, X_2$ 2가지를 쓸 때보다 Loss값이 높게 분포, 성능이 좋지 않음

## 2개 선택

- 2가지 피쳐 형태의 여부(제곱, sin함수)에 따른 영향성을 보고자 하였다.
- $X_1^2, X_2^2$
- $sin(X_1), sin(X_2)$
- $X_1, X_2$(Baseline)
- 결과물 캡쳐
   

| Loss | Train | Test |
| --- | --- | --- |
| $X_1^2, X_2^2$ | 0.045 | 0.042 |
| $sin(X_1), sin(X_2)$ | 0.043 | 0.041 |
| Baseline($X_1, X_2$) | 0.030 | 0.031 |

> 기본 피쳐를 대체하여 가공된 피쳐를 넣을 경우, Loss값이 증가하게 됨

`기본 피쳐는 필수적으로 사용해야 한다고 판단하고 추후 실험.`

## 기본 피쳐 + 제곱/sin함수의 영향

<aside>
💡 피쳐 수 증가에 따라, 정보의 병목으로 인한 손실 영향성을 없애기 위해, **L1의 units을 2→ 6으로 증가**

</aside>

- $X_1, X_2$(Baseline)
- $X_1, X_2, X_1^2, X_2^2$,
- $X_1, X_2, sin(X_1), sin(X_2)$
- $X_1, X_2, X_1X_2$


| Loss | Train | Test |
| --- | --- | --- |
| Baseline($X_1, X_2$) | 0.015 | 0.017 |
| $X_1, X_2, X_1^2, X_2^2$ | 0.033 | 0.033 |
| $X_1, X_2, sin(X_1), sin(X_2)$ | 0.014 | 0.017 |
| $X_1, X_2, X_1X_2$ | 0.013 | 0.013 |

<aside>
💡 $X_1, X_2, X_1^2, X_2^2$의 경우 오히려 일부 학습이 잘 되지 않는 것 같은 경향을 확인(0.033)

</aside>

## 산출물

✅ 확실히 나눌 수 있는 조합
- 베이스라인 + 추가 피쳐엔지니어링 된 산출물
($X_1, X_2, X_1X_2$), ($X_1, X_2, sin(X_1), sin(X_2)$), ($X_1, X_2$)

⛔ 나누지 못하는 조합

- 피쳐를 1개만 사용 그룹($X_1, X_1^2, sin(X_1)$),

## ($X_1, X_2, X_1X_2$) 사용 시, LR을 0.003 (1/10)으로 감소하고 테스트 진행

- LR의 감소 시, 같은 loss값(약 0.014 내외)에 도달하는 데에 소요되는 epoch가 늘어나는 경향성을 확인할 수 있었다.(1500 → 4000)


# 하이퍼 파라미터를 수정해보고 다음을 살펴봅니다.

- 3-1. 레이어
- 3-2. LR
- 3-3. Activation

# 레이어: (units, 8개 정도..) - 주현

- L1 : 1 / 2 / 4 / 8 (L2 5 고정)
- L2 : 3 / 5 / 7 (L1 2 고정)
1. 레이어가 깊어질수록 어떤 특징이 있는지 살펴봅니다. 
    
    **→ 모델의 레이어가 깊어져도 유의미한 성능향상이 없었고, 오히려 너무 깊어지면 로스가 크게 튀는걸 볼 수 있음** 
    
    **3층 : 2-5-5**
    
    유의미한 차이 없음
  
    **5층 2-5-5-5-5**
    
    유의미한 차이 없음
    
    
    
    **4층 2-5-5-5**
    
    유의미한 차이 없음
    
 
    
    **6층 2-5-5-5-5-5**
    
    성능은 좋으나, 로스가 많이 진동하는 것을 보아 과적합이 이루어졌을 수 있음 
    
   
    
2. 레이어가 넓어질수록 어떤 특징이 있는지 살펴봅니다.
    
    **→ 레이어가 넓어질 수록 성능이 조금 올라가지만, 과적합에 유의해야 함** 
    
    1. L1 변경
        
        
        **2층 1-5**
        
        성능 악화 (베이스 0.027 즈음)
        
    
        **2층 8-5**
        
        약간의 성능 증가, 로스 조금 진동
        
       
        **2층 4-5**
        
        약간의 성능 증가, 로스 조금 진동
        
    
        
    2. L2 변경
        
        
        **2층 2-3**
        
        성능 악화 (베이스 0.027 즈음)
        
        
        
        **2층 2-7**
        
        수렴되지 않고 로스가 크게 튐 
        
     
        
    
3. 왜 레이어를 변경할때마다 패턴이 다르게 시작하는지 생각해봅니다
    
    모델의 구조에 따라 특성을 반영하는 측면이 변화하기 때문에
    

# learning rate (11개) - 재현

10^1, 0, -1, -2, -3, -4, -5
3 * 10^0, -1, -2, -3

```
- learning rate가 높아질 때 어떤 특징이 있는지 살펴봅니다.
- learning rate가 낮아질 때 어떤 특징이 있는지 살펴봅니다.
```

[실험 기록](https://www.notion.so/550e94a650de4f7b900787773ee91bfb?pvs=21)

| Learning Rate | 기초 모델 테스트 로스 |
| --- | --- |
| 0.00001 | 0.040 |
| 0.0001 | 0.035 |
| 0.001 | 0.030 |
| 0.003 | 0.030 |
| 0.01 | 0.028 |
| 0.03 | 0.026 |
| 0.1 | 0.030 |

정리:

실험 기록을 봐야 더 제대로 볼 수 있지만, 간단하게 보면 학습이 멈추는 에폭은 학습률 & 모델의 복잡도 등 여러 요인들에 의해 기하급수적으로 바뀌어서 매번 다른 학습률을 지정하여 여러 환경마다 테스트 해보는 것이 필요로 해보임

# activation function (4개) - 서우

- activation function이 달라질 때 어떤 특징이 있는지 살펴봅니다.
- 특히 tanh, sigmoid는 함수의 모양이 비슷한데 수렴속도가 다른 이유를 설명해봅시다.

```
ReLU; epoch 163 정도에 수렴 0.034
tanh; epoch 45 수렴 0.034
sigmoid: epoch 380이 수렴 0.043? 하는거 같아 보였지만 오래 걸리는 것이었음
linear: 13?? 0.046. 이후 업뎃이 잘 안됨 

수렴속도로 치면linear > tanh > ReLU > sigmoid
loss값은 0.034정도에서 수렴된다. linear function은 예외적으로 안된다
=> linear function으로 잘 나눠지는 데이터가 아닌거 아닐까? 비선형 구조

tanh 가 sigmoid보다 훨씬 빨랐다. 그 이유는

1) 시그모이드 함수는 오파역전파 수행 시 기울기가 소실되는 문제가 발생
   입력의 절대값이 크면 0~1로 수렴하면서 그래디언트를 소멸시켜버리기 때
2) 탄젠트함수는 결과값이 (-1,1)이고, 중심값이 0으로 양수와 음수가
   나오는 비중이 비슷해 zigzag현상이 덜하다
   
 원래는 알려진 속도가 ReLU > sigmoid > tanh인거 같은데 (tanh이 연산이 많아서)
 왜 여기서는 다르게 나올까? 중간에 기울기가 소실되었나,, 
 
 - ReLU는 learning rate이 커질수록 (<=0.3) 학습이 빨라지지만,
 학습률이 1 이상이 되면 가중치가 크게 변동되어서 발산.. 
 
```



## 한계점

```
1. 시드 고정이 안되어 같은 값을 넣고 학습을 하여도 매번 다른 결과가 나옴
2. 데이터 역시 랜덤성으로 사람마다 완전히 동일한 데이터를 사용할 수 없었음 
3. epoch의 정확한 고정이 안되어 비교에 어려움이 있음
```